# 12.1 내장 자료형의 상속은 까다롭다

- 사용자가 오버라이드한 코드를 호출하지 않음

In [1]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [2]:
dd = DoppelDict(one=1)
dd

{'one': 1}

In [3]:
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [4]:
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

In [5]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

In [6]:
ad = AnswerDict(a='foo')
ad['a']

42

In [7]:
d = {}
d.update(ad)
d['a']

'foo'

In [8]:
d

{'a': 'foo'}

- collections.Userdict 상속하면 해결 가능

In [9]:
import collections

In [10]:
class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [11]:
dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [12]:
dd.update(three=3)
dd

{'one': [1, 1], 'three': [3, 3]}

In [13]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42

In [14]:
ad = AnswerDict2(a='foo')
ad['a']

42

In [15]:
d = {}
d.update(ad)
d['a']
d

{'a': 42}

# 12.2 다중 상속과 메서드 결정 순서

In [16]:
class A:
    def ping(self):
        print('ping:', self)
        
class B(A):
    def pong(self):
        print('pong', self)
    
class C(A):
    def pong(self):
        print('PONG', self)
        
class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping', self)
        
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [17]:
a = A()
a.ping()

ping: <__main__.A object at 0x000001EB618475E0>


In [18]:
b = B()
b.ping()

ping: <__main__.B object at 0x000001EB618476D0>


In [19]:
b.pong()

pong <__main__.B object at 0x000001EB618476D0>


In [20]:
c = C()
c.pong()

PONG <__main__.C object at 0x000001EB61847460>


In [21]:
c.ping()

ping: <__main__.C object at 0x000001EB61847460>


In [22]:
d = D()
d.pong()

pong <__main__.D object at 0x000001EB61847130>


In [23]:
d.ping()

ping: <__main__.D object at 0x000001EB61847130>
post-ping <__main__.D object at 0x000001EB61847130>


In [24]:
d.pingpong()

ping: <__main__.D object at 0x000001EB61847130>
post-ping <__main__.D object at 0x000001EB61847130>
ping: <__main__.D object at 0x000001EB61847130>
pong <__main__.D object at 0x000001EB61847130>
pong <__main__.D object at 0x000001EB61847130>
PONG <__main__.D object at 0x000001EB61847130>


In [25]:
from diamond import *
d = D()
d.pong()

pong <__main__.D object at 0x000001EB617E5E50>


In [26]:
C.pong(d)

PONG <__main__.D object at 0x000001EB617E5E50>


In [27]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

# 12.3 실세계에서의 다중 상속

Tkinter 가 무엇일까
- Tkinter는 Tcl/Tk에 대한 파이썬 Wrapper로서 Tcl/Tk를 파이썬에 사용할 수 있도록 한 Lightweight GUI 모듈이다
- Tcl : Tool Command Language
- Tk : 크로스 플랫폼에 사용되는 일종의 GUI 툴킷

- 단점 : 타 GUI 프레임워크나 툴킷에 비해 지원되는 위젯들이 부족하고 UI도 그렇게 예쁘지 않다
- 장점 : Python 설치시 기본적으로 내장되어 있는 파이썬 표준 라이브러리이기 때문에 쉽고 간단한 GUI 프로그램을 만들 때 활용

책 설명
- tkinter 모듈에 구현된 Tkinter GUI 툴킷은 표준 라이브러리에서 다중 상속을 극단적으로 사용하는 예를 보여줌 (그림 12-2와 같이)
- Tkinter는 20년이나 되었고, 다중 상속의 단점을 이해하지 못할 때의 예시를 잘 보여줌

간단히 배워보자 (신기함)
- pack : 윈도우 스크린에 보이게 하기

In [ ]:
from tkinter import *
root = Tk()

myLabel = Label(root, text="테스트 라벨 위젯")
myLabel.pack()

mainloop()

- button 위젯 

In [ ]:
from tkinter import *
root = Tk()

# 기본 버튼
btn1 = Button(root, text='click')
btn1.pack()

# 비활성화
btn2 = Button(root, text='click', state=DISABLED)
btn2.pack()

# 50x50 사이즈
btn3 = Button(root, text='click', padx=50, pady=50)
btn3.pack()

mainloop()

- button 위젯 (클릭)

In [29]:
from tkinter import *
root = Tk()

# 클릭 시 실행 함수
def clickEvent():
    label = Label(root, text='여러분과 스터디를 함께 할 수 있어 영광입니다')
    label.pack()

# 기본 버튼
btn = Button(root, text='click', fg='blue', bg='cyan', command=clickEvent)
btn.pack()

mainloop()

- 실행 파일로 만들어서 실행해보기
https://doomed-lab.tistory.com/46

In [32]:
from tkinter import *

def click(key):
    if key == '=':       # '='버튼이면 수식을 계산하여 결과를 표시한다
        try:
            result = eval(entry.get())
            entry.delete(0, END)
            entry.insert(END, str(result))
        except:
            entry.insert(END, "오류!!!!!")
    elif key == 'C':
        entry.delete(0, END)
    else:
        entry.insert(END, key)


window = Tk()
window.title("간단한 계산기")

buttons = ['7', '8', '9', '+', 'C',
           '4', '5', '6', '-', ' ',
           '1', '2', '3', '*', ' ',
           '0', '.', '=', '/', ' ']

#반복문으로 버튼을 생성한다.
i = 0
for b in buttons:
    cmd = lambda x=b: click(x)
    but = Button(window, text=b, width=5, relief='ridge', command=cmd)
    but.grid(row=i//5+1, column=i%5)
    i += 1

#엔트리 위젯은 맨 윗줄의 5개의 셀에 걸쳐서 배치된다
entry = Entry(window, width=33, bg='yellow')
entry.grid(row=0, column=0, columnspan=5)

window.mainloop()